In [12]:
import importlib
import utils
importlib.reload(utils)
from utils import *
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


In [2]:
conso_tou = pd.read_csv("conso_heat_perif_toulouse")
conso_zur = pd.read_csv("conso_heat_perif_zurich")
conso_sev = pd.read_csv("conso_cool_perif_seville")
occ=pd.read_csv("Occupancy_per_hour",delimiter="\t")

In [3]:
files = {
    "agen": "Meteo_Perif_Toulouse_Contemporain/Agen/Simulation_Outputs",
    "albi": "Meteo_Perif_Toulouse_Contemporain/Albi/Simulation_Outputs",
    "auch": "Meteo_Perif_Toulouse_Contemporain/Auch/Simulation_Outputs",
    "toulouse": "Meteo_Perif_Toulouse_Contemporain/Toulouse/Simulation_Outputs",
    "Birmensdorf":"Meteo_Perif_Zurich_Contemporain/Birmensdorf/Simulation_Outputs",
    "Taenikon":"Meteo_Perif_Zurich_Contemporain/Taenikon/Simulation_Outputs",
    "Zurich_fluntern":"Meteo_Perif_Zurich_Contemporain/Zuerich_Fluntern/Simulation_Outputs",
    "Zurich_kloten":"Meteo_Perif_Zurich_Contemporain/Zuerich_kloten/Simulation_Outputs"
}

files2 = {
    "Cordoba": "Meteo_Perif_Seville_Contemporain/Cordoba/Simulation_Outputs",
    "Granada": "Meteo_Perif_Seville_Contemporain/Granada/Simulation_Outputs",
    "Malaga": "Meteo_Perif_Seville_Contemporain/Malaga/Simulation_Outputs",
    "Sevilla": "Meteo_Perif_Seville_Contemporain/Sevilla/Simulation_Outputs"   
}

files3 = {
    "agen": "Meteo_Perif_Toulouse_Contemporain/Agen/Meteo_input",
    "albi": "Meteo_Perif_Toulouse_Contemporain/Albi/Meteo_input",
    "auch": "Meteo_Perif_Toulouse_Contemporain/Auch/Meteo_input",
    "toulouse": "Meteo_Perif_Toulouse_Contemporain/Toulouse/Meteo_input"
}

In [4]:
for city, path in files.items():
      globals()[f"Text_{city}"] = extract_columns(files[city],1)

for city2, path2 in files2.items():
      globals()[f"Text_{city2}"] = extract_columns(files2[city2],1)    

for city3, path3 in files3.items():
      globals()[f"hum_{city3}"] = extract_columns(files3[city3],3)  

occupation=extract_columns("Occupancy_per_hour",1)  

In [5]:
Text_combined_tou = pd.concat([Text_agen, Text_albi,Text_auch,Text_toulouse], axis=0).reset_index(drop=True)
Text_combined_zur = pd.concat([Text_Birmensdorf,Text_Taenikon,Text_Zurich_fluntern,Text_Zurich_kloten], axis=0).reset_index(drop=True)
Text_combined_sev = pd.concat([Text_Cordoba, Text_Granada,Text_Malaga,Text_Sevilla], axis=0).reset_index(drop=True)
Text_combined_tou['clusters'] = conso_tou['clusters']
Text_combined_tou['heat_on'] = conso_tou['heat_on']
Text_combined_zur['clusters'] = conso_zur['clusters']
Text_combined_zur['heat_on'] = conso_zur['heat_on']
Text_combined_sev['clusters'] = conso_sev['clusters']
Text_combined_sev['cool_on'] = conso_sev['cool_on']
Text_combined_tou.columns = Text_combined_tou.columns.astype(str)
Text_combined_zur.columns = Text_combined_zur.columns.astype(str)
Text_combined_sev.columns = Text_combined_sev.columns.astype(str)


Text_occ_agen = pd.concat([Text_agen,occupation],axis=1).reset_index(drop=True)
Text_occ_albi = pd.concat([Text_albi,occupation],axis=1).reset_index(drop=True)
Text_occ_auch = pd.concat([Text_auch,occupation],axis=1).reset_index(drop=True)
Text_occ_toulouse = pd.concat([Text_toulouse,occupation],axis=1).reset_index(drop=True)
Text_occ_combined_tou = pd.concat([Text_occ_agen, Text_occ_albi,Text_occ_auch,Text_occ_toulouse], axis=0).reset_index(drop=True)
Text_occ_combined_tou['clusters'] = conso_tou['clusters']
Text_occ_combined_tou['heat_on'] = conso_tou['heat_on']
Text_occ_combined_tou.columns = Text_occ_combined_tou.columns.astype(str)


Hum_combined_tou = pd.concat([hum_agen, hum_albi,hum_auch,hum_toulouse], axis=0).reset_index(drop=True)
Hum_combined_tou['clusters'] = conso_tou['clusters']
Hum_combined_tou['heat_on'] = conso_tou['heat_on']
Hum_combined_tou.columns = Hum_combined_tou.columns.astype(str)


In [6]:
Text_combined_tou

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,clusters,heat_on
0,10.40,10.45,10.40,10.25,10.20,10.15,10.10,10.10,10.15,10.50,...,13.70,13.10,12.60,12.25,11.95,11.60,11.25,10.90,2.0,1
1,10.45,9.80,9.15,8.75,8.45,8.10,7.80,7.55,7.40,7.45,...,9.60,9.15,8.75,8.45,8.20,7.95,7.65,7.15,2.0,1
2,7.05,7.30,7.30,7.30,7.35,7.45,7.50,7.55,7.60,7.85,...,12.20,11.65,11.10,10.70,10.30,9.80,9.35,8.75,2.0,1
3,8.30,8.00,7.35,6.85,6.55,6.25,5.95,5.70,5.60,6.00,...,9.85,9.20,8.45,7.75,7.10,6.45,5.80,5.15,0.0,1
4,4.55,4.00,3.45,3.05,2.75,2.45,2.25,2.10,2.00,3.00,...,11.10,9.70,8.65,8.10,7.55,7.05,6.50,5.95,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-3.25,-3.25,-3.30,-3.35,-3.45,-3.50,-3.55,-3.60,-3.60,-3.00,...,1.10,0.30,-0.30,-0.55,-0.80,-1.00,-1.20,-1.45,0.0,1
1456,-1.55,-1.40,-1.25,-1.15,-1.05,-0.95,-0.85,-0.75,-0.65,0.15,...,6.10,5.20,4.60,4.45,4.30,4.15,4.00,3.85,0.0,1
1457,3.60,3.15,2.70,2.40,2.15,1.90,1.70,1.50,1.45,2.20,...,7.95,7.00,6.35,6.30,6.30,6.25,6.20,6.20,2.0,1
1458,6.25,6.40,6.55,6.65,6.80,6.95,7.05,7.15,7.20,7.55,...,10.45,10.05,9.45,9.20,9.20,8.95,8.60,8.30,2.0,1


In [23]:
from tensorflow.keras.utils import to_categorical


X=Text_combined_tou.drop(columns=["clusters"])
y=Text_combined_tou["clusters"]
X_scaled=standardize_data(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4) 

In [24]:


def f1_score(y_true, y_pred):
    precision = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1))) / (tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1))) + tf.keras.backend.epsilon())
    recall = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1))) / (tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1))) + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))



In [29]:
model = Sequential()
model.add(Dense(units=120, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=60, activation='relu'))
model.add(Dense(units=4, activation='softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 120)                 │           3,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 60)                  │           7,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 4)                   │             244 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,624 (41.50 KB)

 Trainable params: 10,624 (41.50 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_score])

In [31]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6611 - f1_score: 0.4771 - loss: 0.8903 - val_accuracy: 0.8493 - val_f1_score: 0.6682 - val_loss: 0.5449
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8144 - f1_score: 0.7094 - loss: 0.5075 - val_accuracy: 0.8596 - val_f1_score: 0.8185 - val_loss: 0.4086
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8428 - f1_score: 0.8204 - loss: 0.3675 - val_accuracy: 0.8767 - val_f1_score: 0.8281 - val_loss: 0.3434
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8638 - f1_score: 0.8455 - loss: 0.3256 - val_accuracy: 0.8596 - val_f1_score: 0.8812 - val_loss: 0.3037
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8520 - f1_score: 0.8532 - loss: 0.3012 - val_accuracy: 0.8459 - val_f1_score: 0.8553 - val_loss: 0.3203
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8506 - f1_score: 0.8536 - loss: 0.2889 - val_accuracy: 0.8767 - val_f1_score: 0.8766 - val_loss

In [34]:
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D, Dropout, BatchNormalization, InputLayer

X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [35]:
model_cnn = Sequential([
    InputLayer(input_shape=(24, 1)), 
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),

    Flatten(),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax') 
])


model_cnn.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy',f1_score])


history_cnn = model_cnn.fit(X_train_cnn, y_train, 
                            validation_data=(X_test_cnn, y_test), 
                            epochs=50, batch_size=32)

Epoch 1/50


C:\Users\toubia\anaconda3\envs\notebook\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.5623 - f1_score: 0.5379 - loss: 1.1299 - val_accuracy: 0.6575 - val_f1_score: 0.0000e+00 - val_loss: 1.1852
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7504 - f1_score: 0.7467 - loss: 0.5714 - val_accuracy: 0.3767 - val_f1_score: 0.0000e+00 - val_loss: 1.1845
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7372 - f1_score: 0.7213 - loss: 0.5673 - val_accuracy: 0.3048 - val_f1_score: 0.0000e+00 - val_loss: 1.1782
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7379 - f1_score: 0.7183 - loss: 0.5789 - val_accuracy: 0.3973 - val_f1_score: 0.0360 - val_loss: 1.1095
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7471 - f1_score: 0.7415 - loss: 0.5655 - val_accuracy: 0.5993 - val_f1_score: 0.0627 - val_loss: 1.0924
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7399 - f1_score: 0.7244 - loss: 0.6101 - val_accuracy: 0.3973 - val_f1_score: 0.1353 - val_l